In [ ]:
import geopandas as gpd
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
from tqdm import tqdm
from srai.joiners import IntersectionJoiner
from sklearn.decomposition import PCA
from collections import defaultdict, deque

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Constants
RESOLUTION = 10
EMBEDDING_DIM = 200     # embedding dim equal to urban2vec & m3g study
NUM_EPOCHS = 400       # Increased number of epochs
BATCH_SIZE = 64         # Increased batch size
LEARNING_RATE = 5e-6    # nice and slow
CHECKPOINT_DIR = f'checkpoints_urban2vec_res{RESOLUTION}_dim{EMBEDDING_DIM}'

In [ ]:
# # prepare poi unbuffered features for export
# from srai.loaders import OSMPbfLoader
# from srai.loaders.osm_loaders.filters import GEOFABRIK_LAYERS
# 
# tags = GEOFABRIK_LAYERS
# loader = OSMPbfLoader()
# regions_gdf = gpd.read_file(f"selected_regions_{RESOLUTION}.geojson").set_index("region_id")
# features_gdf = loader.load(regions_gdf, tags)
# 
# # Convert geometry to WKT for Parquet storage
# features_gdf['geometry'] = features_gdf['geometry'].apply(lambda geom: geom.wkt)
# # Export to Parquet
# features_gdf.to_parquet(f"POI_features_unbuffered_{RESOLUTION}.parquet", index=True)
# print(f"Exported POI features to POI_features_unbuffered_{RESOLUTION}.parquet")

Word2vec-style approach:
In this approach, we treat each POI in a region as an individual "word" in the "sentence" (region). The model learns to predict the context (other POIs in the region) given a target POI, or vice versa. Negative sampling based on frequency from the entire corpus of words (not just local hexagons). This approach is more similar hex2vec than 2step ring sampling. 

In [ ]:
class CircleLoss(nn.Module):
    def __init__(self, m=0.25, gamma=256):
        super().__init__()
        self.m, self.gamma = m, gamma
        self.soft_plus = nn.Softplus()

    def forward(self, anchor, positive, negative):
        sp = torch.sum(anchor * positive, dim=1)
        sn = torch.sum(anchor * negative, dim=1)

        ap = torch.clamp_min(-sp.detach() + 1 + self.m, min=0.)
        an = torch.clamp_min(sn.detach() + self.m, min=0.)

        delta_p, delta_n = 1 - self.m, self.m
        logit_p = -ap * (sp - delta_p) * self.gamma
        logit_n = an * (sn - delta_n) * self.gamma

        return self.soft_plus(logit_n + logit_p).mean()

In [ ]:
class Urban2VecDataset(Dataset):
    def __init__(self, regions_gdf, joint_gdf, features_df):
        # Create region dictionary
        self.regions = list(regions_gdf.index)
        self.region_to_idx = {r: i for i, r in enumerate(self.regions)}

        # Collect all POIs across all features
        all_pois = set()
        for poi_list in features_df['poi_list']:
            all_pois.update(poi_list)  # Add unique POIs from each list

        # Create POI dictionary
        self.poi_dictionary = sorted(all_pois)
        self.poi_to_idx = {poi: idx for idx, poi in enumerate(self.poi_dictionary)}

        self.num_pois = len(self.poi_dictionary)  # Store num_pois as attribute

        # Create region to POIs mapping, filtering POIs not in poi_dictionary
        self.region_to_pois = defaultdict(list)
        for (region_id, feature_id) in joint_gdf.index:
            region_idx = self.region_to_idx[region_id]
            poi_list = features_df.loc[feature_id, 'poi_list']
            filtered_poi_list = [self.poi_to_idx[poi] for poi in poi_list if poi in self.poi_to_idx]
            self.region_to_pois[region_idx].extend(filtered_poi_list)  # Add filtered list

        # Create list of valid regions (regions with POIs)
        self.valid_regions = [idx for idx, pois in self.region_to_pois.items() if pois]

        # Printing for debugging
        print(f"Number of valid regions: {len(self.valid_regions)}")
        print(f"Number of unique POIs: {len(self.poi_dictionary)}")

        # Calculate POI frequencies for negative sampling
        poi_counts = np.zeros(len(self.poi_dictionary))
        for pois in self.region_to_pois.values():
            for poi in pois:
                poi_counts[poi] += 1
        self.neg_sample_dist = np.power(poi_counts, 0.75)
        self.neg_sample_dist /= self.neg_sample_dist.sum()

    def __len__(self):
        return len(self.valid_regions)

    def __getitem__(self, idx):
        region_idx = self.valid_regions[idx]
        pois = self.region_to_pois[region_idx]

        # assert pois, f"Region {region_idx} has no valid POIs."  # Add assertion

        positive_poi = np.random.choice(pois)
        negative_poi = np.random.choice(len(self.poi_dictionary), p=self.neg_sample_dist) % self.num_pois

        # # Print statements for debugging
        # print(f"Region index: {region_idx}, Max region index: {self.get_num_regions() - 1}")
        # print(f"Positive POI index: {positive_poi}, Max POI index: {self.get_num_pois() - 1}")
        # print(f"Negative POI index: {negative_poi}, Max POI index: {self.get_num_pois() - 1}")
        # 
        # assert 0 <= positive_poi < self.num_pois, f"Invalid positive POI index: {positive_poi}"
        # assert 0 <= negative_poi < self.num_pois, f"Invalid negative POI index: {negative_poi}"

        return region_idx, positive_poi, negative_poi

    def get_num_regions(self):
        return len(self.regions)

    def get_num_pois(self):
        return len(self.poi_dictionary)

In [ ]:
def custom_collate(batch):
    region_idx, positive_poi, negative_poi = zip(*batch)
    return (
        torch.tensor(region_idx, dtype=torch.long),
        torch.tensor(positive_poi, dtype=torch.long),
        torch.tensor(negative_poi, dtype=torch.long)
    )

In [ ]:
class Urban2VecModel(nn.Module):
    def __init__(self, num_regions, num_pois, embedding_dim):
        super(Urban2VecModel, self).__init__()
        self.region_embedding = nn.Embedding(num_regions, embedding_dim)
        self.poi_embedding = nn.Embedding(num_pois, embedding_dim)

    def forward(self, region_indices, poi_indices):
        region_embed = self.region_embedding(region_indices)
        poi_embed = self.poi_embedding(poi_indices)
        return region_embed, poi_embed

In [ ]:
def load_data():
    regions_gdf = gpd.read_file(f"selected_regions_{RESOLUTION}.geojson").set_index("region_id")
    features_df = pd.read_parquet(f"POI_features_unbuffered_{RESOLUTION}.parquet")
    
    # use features_df NOT gdf. Filter out columns with POI catagories and get all non None values.
    poi_columns = [col for col in features_df.columns if col != 'geometry']
    features_df['poi_list'] = features_df[poi_columns].apply(lambda row: [val for val in row if pd.notna(val) and val != 'None'], axis=1)
    features_df = features_df[features_df['poi_list'].map(len) > 0]
    
    # Make features_gdf from features_df by adding geometry
    features_gdf = gpd.GeoDataFrame(
        features_df,
        geometry=gpd.GeoSeries.from_wkt(features_df['geometry']),
        crs=regions_gdf.crs
    )

    # use IntersectionJoiner to join regions_gdf and features_gdf for sampling later
    joiner = IntersectionJoiner()
    joint_gdf = joiner.transform(regions_gdf, features_gdf)

    return regions_gdf, joint_gdf, features_df

In [ ]:
def train_urban2vec(model, dataloader, optimizer, criterion, device, num_epochs):
    model.train()
    loss_values = []
    avg_loss_queue = deque(maxlen=10)  # Queue to store last 10 epoch losses

    with tqdm(total=len(dataloader) * num_epochs, desc="Training", unit="batch") as pbar:
        for epoch in range(num_epochs):
            total_loss = 0
            for batch in dataloader:  # No tqdm here
                region_idx, positive_poi, negative_poi = batch
                region_idx, positive_poi, negative_poi = region_idx.to(device), positive_poi.to(device), negative_poi.to(device)

                optimizer.zero_grad()
                region_embed = model.region_embedding(region_idx)
                positive_embed = model.poi_embedding(positive_poi)
                negative_embed = model.poi_embedding(negative_poi)

                loss = criterion(region_embed, positive_embed, negative_embed)
                loss.backward()
                optimizer.step()

                total_loss += loss.item()
                loss_values.append(loss.item())

                # Update progress bar after each batch
                pbar.update(1)

            # Calculate and display average epoch loss
            avg_epoch_loss = total_loss / len(dataloader)
            avg_loss_queue.append(avg_epoch_loss)

            # Calculate 10-epoch running average
            running_avg_loss = sum(avg_loss_queue) / len(avg_loss_queue)
            pbar.set_postfix(loss=running_avg_loss)  # Show running average in tqdm

    return model, loss_values

In [ ]:
def apply_pca(embeddings, n_components):
    pca = PCA(n_components=n_components)
    return pca.fit_transform(embeddings)

In [ ]:
def load_pretrained_embeddings(regions, embedding_file, n_components=EMBEDDING_DIM):
    pretrained_embeddings = pd.read_csv(embedding_file, index_col=0)
    pretrained_embeddings = pretrained_embeddings.reindex(regions)  # Align with current regions
    pretrained_embeddings = pretrained_embeddings.fillna(0)  # Fill missing values

    # Check if there are enough embeddings for PCA
    if pretrained_embeddings.shape[1] >= n_components:
        reduced_embeddings = apply_pca(pretrained_embeddings.values, n_components=n_components)
    else:
        print("Not enough pre-trained embeddings for PCA. Using as is.")
        reduced_embeddings = pretrained_embeddings.values

    return torch.tensor(reduced_embeddings, dtype=torch.float32)

In [ ]:
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    regions_gdf, joint_gdf, features_df = load_data()

    dataset = Urban2VecDataset(regions_gdf, joint_gdf, features_df)
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=custom_collate)

    num_regions = dataset.get_num_regions()
    num_pois = dataset.get_num_pois()
    model = Urban2VecModel(num_regions, num_pois, EMBEDDING_DIM).to(device)

    # Load pre-trained embeddings with proper handling and debugging prints
    embedding_file = f"embeddings_aerial_{RESOLUTION}_finetune.csv"
    try:
        print(f"Loading pretrained embeddings from: {embedding_file}")
        pretrained_embeddings = load_pretrained_embeddings(dataset.regions, embedding_file)
        print(f"pretrained_embeddings shape: {pretrained_embeddings.shape}")

        model.region_embedding.weight.data.copy_(pretrained_embeddings)
        print("Loaded and PCA-reduced/used as is pre-trained region embeddings.")
    except FileNotFoundError:
        print("Pre-trained embeddings not found. Initializing randomly.")

    # Print for debugging
    print(f"Region embeddings shape: {model.region_embedding.weight.shape}")

    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    criterion = CircleLoss()

    os.makedirs(CHECKPOINT_DIR, exist_ok=True)
    loss_values = []  # Initialize loss_values here
    try:
        model, loss_values = train_urban2vec(model, dataloader, optimizer, criterion, device, NUM_EPOCHS)
    except RuntimeError as e:
        print(f"RuntimeError during training: {e}")
        print("Consider setting CUDA_LAUNCH_BLOCKING=1 for more detailed error messages.")

    region_embeddings = model.region_embedding.weight.detach().cpu().numpy()
    poi_embeddings = model.poi_embedding.weight.detach().cpu().numpy()

    pd.DataFrame(region_embeddings, index=dataset.regions).to_csv(os.path.join(CHECKPOINT_DIR, 'step2_region_embeddings.csv'))
    pd.DataFrame(poi_embeddings, index=dataset.poi_dictionary).to_csv(os.path.join(CHECKPOINT_DIR, 'step2_poi_embeddings.csv'))

    print("Training completed and embeddings saved.")
    return loss_values

In [ ]:
if __name__ == "__main__":
    loss_values = main()

In [ ]:
# plot loss values and add a smoothened average line
import matplotlib.pyplot as plt
plt.plot(loss_values)
plt.plot(pd.Series(loss_values).rolling(1000).mean())
plt.xlabel("Iterations")
plt.ylabel("Loss")
plt.title("Circle Loss")
plt.show()

In [ ]:
# import trained region embeddings
region_embeddings = pd.read_csv(os.path.join(CHECKPOINT_DIR, 'step2_region_embeddings.csv'), index_col=0)
regions_gdf = gpd.read_file(f"selected_regions_{RESOLUTION}.geojson").set_index("region_id")

In [ ]:
from Plotting import pca_plot, cluster_agglomerative_plot, cluster_kmeans_plot
import warnings
warnings.filterwarnings("ignore")
cluster_agglomerative_plot(region_embeddings, regions_gdf, 10)

In [ ]:
# poi_dict 

In [ ]:
# features_test_gdf = pd.read_parquet(f"POI_features_unbuffered_{RESOLUTION}.parquet")
# features_test_gdf.head()

In [ ]:
# def resume_training(checkpoint_path, model, optimizer, scheduler, start_epoch):
#     if os.path.exists(checkpoint_path):
#         checkpoint = torch.load(checkpoint_path)
#         model.load_state_dict(checkpoint['model_state_dict'])
#         optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#         if 'scheduler_state_dict' in checkpoint:
#             scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
#         else:
#             print("Scheduler state not found in checkpoint. Using default scheduler.")
#         start_epoch = checkpoint['epoch']
#         print(f"Resuming training from epoch {start_epoch}")
#     else:
#         print("No checkpoint found. Starting training from scratch.")
#     return model, optimizer, scheduler, start_epoch
# 
# # Usage:
# checkpoint_path = os.path.join(CHECKPOINT_DIR, 'checkpoint_epoch_22.pth')  # or 'best_model.pth'
# model, optimizer, scheduler, start_epoch = resume_training(checkpoint_path, model, optimizer, scheduler, 0)

In [ ]:
# # plotting embeddings for test
# from Plotting import pca_plot, cluster_agglomerative_plot, cluster_kmeans_plot
# import warnings
# warnings.filterwarnings("ignore")
# from sklearn.decomposition import PCA
# pca = PCA(n_components=100)
# embeddings_reduced_df = pca.fit_transform(embedding_df)
# # print variance explained
# print(f"Variance explained: {pca.explained_variance_ratio_.sum()*100}%")
# # ensure the index is preserved
# embeddings_reduced_df = pd.DataFrame(embeddings_reduced_df, index=embedding_df.index)
# cluster_agglomerative_plot(embeddings_reduced_df, regions_gdf, 10)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Ensure the regions_gdf and embeddings_aerial, embedding_df are loaded correctly
# Assuming regions_gdf['afw', 'fys', 'onv', 'soc', 'vrz', 'won'] contains the scores

# Get the Leefbaarometer scores
scores_df = regions_gdf[['afw', 'fys', 'onv', 'soc', 'vrz', 'won']]

# Initialize lists to store R-squared values
r2_scores_step1 = []
r2_scores_step2 = []
score_names = scores_df.columns

# Loop over each score
for score_name in score_names:
    scores = scores_df[score_name]

    # Get the embeddings
    embeddings_step1 = pd.read_csv('embeddings_aerial_10_finetune.csv', index_col=0)
    #embeddings_step1 = embeddings_aerial.loc[regions_gdf.index]
    embeddings_step2 = region_embeddings.loc[regions_gdf.index]
    #embeddings_step2 = embedding_df.loc[regions_gdf.index]
    
    # ensure same dimensionality with PCA
    pca = PCA(n_components=30)
    embeddings_step1 = pca.fit_transform(embeddings_step1)
    embeddings_step2 = pca.fit_transform(embeddings_step2)
    
    # Fit the model for step 1
    model_step1 = LinearRegression().fit(embeddings_step1, scores)
    r2_step1 = r2_score(scores, model_step1.predict(embeddings_step1))
    r2_scores_step1.append(r2_step1)

    # Fit the model for step 2
    model_step2 = LinearRegression().fit(embeddings_step2, scores)
    r2_step2 = r2_score(scores, model_step2.predict(embeddings_step2))
    r2_scores_step2.append(r2_step2)

# Plotting the R-squared values
x = np.arange(len(score_names))
width = 0.35

fig, ax = plt.subplots(figsize=(10, 6))
bar1 = ax.bar(x - width/2, r2_scores_step1, width, label='Step 1')
bar2 = ax.bar(x + width/2, r2_scores_step2, width, label='Step 2')

# Adding labels and titles
ax.set_xlabel('Leefbaarometer Scores')
ax.set_ylabel('R-squared Value')
ax.set_title('R-squared Values of Embeddings Predicting Leefbaarometer Scores')
ax.set_xticks(x)
ax.set_xticklabels(score_names)
ax.legend()

# Display the bar chart
plt.tight_layout()
plt.show()

In [ ]:
# # get r squared value of aerial embeddings to leefbaarometer scores in region_gdf (regions_gdf['afw'])
# # get the scores
# scores = regions_gdf['afw']
# # get the embeddings
# 
# # fit the model
# model = LinearRegression().fit(embeddings, scores)
# # get the r squared value
# r2 = r2_score(scores, model.predict(embeddings))
# print(r2)